In [35]:
import json
import pandas as pd
import numpy as np
from bidict import bidict
import pickle

In [2]:
with open("../../data/datasets/title_seq.json", "r") as f:
    title_seq = json.load(f)

In [36]:
with open('../../data/datasets/train_ids.pkl', 'rb') as f:
    train_ids = pickle.load(f)

In [19]:
data = pd.read_pickle("../../data/datasets/top_data.pkl")

In [3]:
title_seq.keys()

dict_keys(['title_to_id', 'train_data', 'test_data'])

In [4]:
train = title_seq['train_data']
title_to_id = title_seq['title_to_id']

In [9]:
title_to_id = bidict(title_to_id)

In [15]:
mapped_train = [[title_to_id.inv[i] for i in seq] for seq in train]

In [69]:
mapped_train[23]

['shift supervisor', 'key holder', 'key holder']

In [72]:
data[data._id == train_ids[23]]

,_id,duration,end_date,function,industry,job_index,mission,place,start_date
11405416,5779c0ec9d15a906688b9f8b,70,NaN,shift supervisor,Retail,0,"Open and closing store, daily and weekly repor...",NaN,2011-08
11405417,5779c0ec9d15a906688b9f8b,13,2011-08,key holder,Retail,1,"- Open and close store, daily paper work, sell...",NaN,2010-07
11405418,5779c0ec9d15a906688b9f8b,6,2010-08,key holder,Retail,2,"Would have to open and close the store, do the...",NaN,2010-02


In [61]:
func = data.groupby('_id')['function'].apply(list)

In [62]:
func_reversed = data.groupby('_id')['function'].apply(lambda x: list(reversed(list(x))))

In [64]:
func["55e74b570b04514a0b94e871"]

['senior software developer',
 'software developer',
 'software developer',
 'software developer']

In [65]:
func_reversed["55e74b570b04514a0b94e871"]

['software developer',
 'software developer',
 'software developer',
 'senior software developer']

In [67]:
t = [[title_to_id[title] for title in func_reversed[i]] for i in train_ids]

In [68]:
[[title_to_id.inv[i] for i in seq] for seq in t]

[['merchandiser',
  'cofounder',
  'sales representative',
  'sales representative',
  'territory manager'],
 ['store manager', 'store manager', 'district manager'],
 ['software developer',
  'software developer',
  'software developer',
  'senior software developer'],
 ['trainee',
  'research assistant',
  'research assistant',
  'postdoctoral fellow'],
 ['software engineer',
  'software engineer',
  'software engineer',
  'software engineer',
  'senior software developer',
  'senior software developer',
  'senior software engineer',
  'chief executive officer'],
 ['independent consultant', 'sales consultant', 'sales consultant'],
 ['line cook', 'chef', 'chef'],
 ['kinesiologist', 'kinesiologist', 'kinesiologist'],
 ['director', 'instructor', 'instructor', 'substitute teacher'],
 ['accountant', 'accountant', 'accountant', 'data entry clerk'],
 ['owner/operator',
  'financial advisor',
  'financial advisor',
  'financial advisor',
  'financial advisor'],
 ['broker', 'broker', 'broker']